In [2]:
import json
import pandas as pd

## Function to split the json

In [22]:
def split_list_to_json(files, data):
    # Calculate the number of items per file
    n = len(data)
    chunk_size = n // 4

    # Split data and write to files
    for i in range(4):
        start_index = i * chunk_size
        # Ensure the last file contains all remaining data in case of uneven division
        if i == 3:
            end_index = n
        else:
            end_index = start_index + chunk_size

        part = data[start_index:end_index]

        # Writing to file
        with open(files[i], 'w') as file:
            json.dump(part, file, indent=4)


## Function to combine json

In [5]:
def combine_json(files):
    combined_data = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as fin:
            # Load data from the file and append it to the combined list
            data = json.load(fin)
            combined_data.extend(data)
    return combined_data

In [4]:

files = ['../data_upload/final_db1.json', '../data_upload/final_db1.json', '../data_upload/final_db1.json', '../data_upload/final_db1.json']
db_raw = combine_json(files)

In [4]:
with open("../data/final_db_pb_tags.json", "r") as fin:
    db_processed = json.load(fin)

In [ ]:
pub_title = {}
for entry in db_raw:
    if entry["_source"]['content_metadata']['language'] == 'en':
        pub_title[entry['_id']] = {}
        pub_title[entry['_id']]['publication_date'] = entry["_source"]['content_metadata']['context']['publication_date'][:10]
        pub_title[entry['_id']]['title_embeddings'] = (entry["_source"]['embeddings']['title_en']['all-mpnet-base-v2-normed']['content'])


In [7]:
for entry in db_processed:
    id = entry['id']
    entry['Title_embeddings'] = pub_title[id]['title_embeddings']
    entry['Publication_date'] = pub_title[id]['publication_date']
    

In [36]:
files = ['../data_upload/final_db1.json', '../data_upload/final_db2.json', '../data_upload/final_db3.json', '../data_upload/final_db4.json']
split_list_to_json(files, db_processed)

In [20]:
def remove_tags_embeddings(articles):
    for article in articles:
        article.pop('phBERT_tags_embeddings', None)  # Remove the key if it exists, do nothing otherwise

remove_tags_embeddings(db_processed)

In [21]:
db_processed[0].keys()

dict_keys(['id', 'Text', 'Title', 'embeddings', 'combined', 'tags', 'phrase_Bert_tags_embeddings', 'Title_embeddings', 'Publication_date'])

In [24]:
files = ['../data_upload/adv_rag/rag_db1.json', '../data_upload/adv_rag/rag_db2.json', '../data_upload/adv_rag/rag_db3.json', '../data_upload/adv_rag/rag_db4.json']
split_list_to_json(files, db_processed)

## PROCESSING NEW DATA FOR URLS

In [6]:
files = ['../data/updated_articles/batch1.json', '../data/updated_articles/batch2.json', '../data/updated_articles/batch3.json', '../data/updated_articles/batch4.json', '../data/updated_articles/batch5.json', '../data/updated_articles/batch6.json']
db_raw = combine_json(files)

In [10]:
db_raw[500]

{'identification': {'drupal': {'id': 'st_1144809'}},
 'content_metadata': {'context': {'content_type': 'article',
   'content_source_system': 'CUE',
   'body_en': "Netherlands midfielder Georginio Wijnaldum has joined Al-Ettifaq from French champions Paris Saint Germain, the Saudi Pro League club said on Saturday.The clubs did not disclose a transfer fee for the 32-year-old who joins former Liverpool team mate Jordan Henderson at Steven Gerrard's side.The announcement came after Al-Ettifaq beat Damac 3-1 to move up to fifth place on 10 points, three behind leaders Al-Hilal and Al-Taawoun.Wijnaldum moved to PSG in 2021 but after struggling to make an impact he joined AS Roma on loan.He broke his leg in training in August 2022, which limited him to 23 games with the Italian side in all competitions, and then returned to Paris.Under new PSG manager Luis Enrique, Wijnaldum had no future in France and ex-Liverpool captain Gerrard persuaded him to join Al-Ettifaq. REUTERS",
   'title_en': "D